# Probando el índice invertido


Como en la práctica anterior, usted puede usar el código del indice invertido. A continuación se carga el código del indice invertido del Tema 3.

In [1]:
include("invindex.jl")

create_inverted_index

## Búsqueda en el índice invertido

In [2]:

"""
    Performs the union of the multiset `res` with the set `plist`
"""
function union!(res, plist)
    for docID in plist
        c = get(res, docID, 0)
        res[docID] = c + 1
    end
end
    
"""
Searching the best ``k`` items. The fitness is computed with the number of shared tokens with the query. It returns an array of pairs
``(documentId, frequency)`` ordered by frequency (in descendent order).
"""
function search(invindex, text; k=10)
    bow = create_bow(text)
    res = Dict{Int,Int}()
    for (term, freq) in bow
        plist = get(invindex, term, nothing)
        if plist != nothing
            union!(res, plist)
        end        
    end
    
    pairs = collect(res)
    sort!(pairs, by=x -> -x.second)
    
    if length(pairs) > k
        @info "selecting $k documents from $(length(pairs))"
        pairs[1:k]
    else
        pairs
    end
end

search

In [3]:
function prepare_collection()
    tweets = load_tweets("politicos.json")
    invindex = create_inverted_index(tweets)
    tweets, invindex
end

tweets, invindex = prepare_collection()
println("base de datos e índice invertido cargados")

base de datos e índice invertido cargados


In [4]:
"""
A simple pretty printing method for objects inside the global dataset
"""
function print_query_and_search(tweets, query; k=10)
    println("### Query:", query, "\n")

    counter = 1
    for (resID, freq) in search(invindex, query, k=k)
        println("== result: $counter, similarity: $freq, text: ", tweets[resID]["text"])
        # println(JSON.json(tweets[resID]), "\n")
        counter += 1
    end
end

print_query_and_search(tweets, tweets[rand(1:length(tweets))]["text"], k=10)


### Query:Le contaremos todo lo que ocurrió en la reunión de AMLO con la @CONAGO_oficial. A las 22:30 en @ImagenTVMex https://t.co/jpaxR5Otdf

== result: 1, similarity: 6, text: Le contaremos todo lo que ocurrió en la reunión de AMLO con la @CONAGO_oficial. A las 22:30 en @ImagenTVMex https://t.co/jpaxR5Otdf
== result: 2, similarity: 4, text: Ciudadana le roba un beso a AMLO. Ocurrió antes de iniciar la reunión de López Obrador con los gobernadores de la @CONAGO_oficial.  ¿De qué se quejó el mandatario @Silvano_A ante el tabasqueño? Sigue esta liga ➡️ https://t.co/ffvNcOwrCW https://t.co/urpXjmpSbg
== result: 3, similarity: 3, text: A las 22:30 en @ImagenTVMex tendremos todos los detalles de la reunión de AMLO con empresarios de la @CONCAMIN. Le explicaremos esto que dijo sobre los gasolinazos https://t.co/W0o2dYi7Gk
== result: 4, similarity: 3, text: @CONAGO_oficial @VelascoM_ @lopezobrador_ #AHORA AMLO asegura que su reunión con la Conago fue de respeto sin reclamos y se acordó traba

┌ Info: selecting 10 documents from 23515
└ @ Main In[2]:30


Este tipo de operaciones pueden ser utilizados para remover o contabilizar duplicados muy similares. Así mismo, ajustando la manera en que se calculan los resultados más pertinentes, es posible utilizar el índice invertido para realizar calculos complejos y costosos de manera eficiente como agrupamiento.